In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier, plot_tree,  _tree
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit, StratifiedShuffleSplit

from joblib import Parallel, delayed

from time import time

import duckdb
import time

import optuna
from optuna.visualization import plot_param_importances, plot_contour,  plot_slice, plot_optimization_history

In [ ]:
# SEEDS 
400009,  500009,  600011,  700021,  800021.

In [4]:
pd.set_option('display.max_columns', None)


,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,mactivos_margen,mpasivos_margen,cproductos,tcuentas,ccuenta_corriente,mcuenta_corriente_adicional,mcuenta_corriente,ccaja_ahorro,mcaja_ahorro,mcaja_ahorro_adicional,mcaja_ahorro_dolares,cdescubierto_preacordado,mcuentas_saldo,ctarjeta_debito,ctarjeta_debito_transacciones,mautoservicio,ctarjeta_visa,ctarjeta_visa_transacciones,mtarjeta_visa_consumo,ctarjeta_master,ctarjeta_master_transacciones,mtarjeta_master_consumo,cprestamos_personales,mprestamos_personales,cprestamos_prendarios,mprestamos_prendarios,cprestamos_hipotecarios,mprestamos_hipotecarios,cplazo_fijo,mplazo_fijo_dolares,mplazo_fijo_pesos,cinversion1,minversion1_pesos,minversion1_dolares,cinversion2,minversion2,cseguro_vida,cseguro_auto,cseguro_vivienda,cseguro_accidentes_personales,ccaja_seguridad,cpayroll_trx,mpayroll,mpayroll2,cpayroll2_trx,ccuenta_debitos_automaticos,mcuenta_debitos_automaticos,ctarjeta_visa_debitos_automaticos,mttarjeta_visa_debitos_automaticos,ctarjeta_master_debitos_automaticos,mttarjeta_master_debitos_automaticos,cpagodeservicios,mpagodeservicios,cpagomiscuentas,mpagomiscuentas,ccajeros_propios_descuentos,mcajeros_propios_descuentos,ctarjeta_visa_descuentos,mtarjeta_visa_descuentos,ctarjeta_master_descuentos,mtarjeta_master_descuentos,ccomisiones_mantenimiento,mcomisiones_mantenimiento,ccomisiones_otras,mcomisiones_otras,cforex,cforex_buy,mforex_buy,cforex_sell,mforex_sell,ctransferencias_recibidas,mtransferencias_recibidas,ctransferencias_emitidas,mtransferencias_emitidas,cextraccion_autoservicio,mextraccion_autoservicio,ccheques_depositados,mcheques_depositados,ccheques_emitidos,mcheques_emitidos,ccheques_depositados_rechazados,mcheques_depositados_rechazados,ccheques_emitidos_rechazados,mcheques_emitidos_rechazados,tcallcenter,ccallcenter_transacciones,thomebanking,chomebanking_transacciones,ccajas_transacciones,ccajas_consultas,ccajas_depositos,ccajas_extracciones,ccajas_otras,catm_trx,matm,catm_trx_other,matm_other,ctrx_quarter,tmobile_app,cmobile_app_trx,Master_delinquency,Master_status,Master_mfinanciacion_limite,Master_Fvencimiento,Master_Finiciomora,Master_msaldototal,Master_msaldopesos,Master_msaldodolares,Master_mconsumospesos,Master_mconsumosdolares,Master_mlimitecompra,Master_madelantopesos,Master_madelantodolares,Master_fultimo_cierre,Master_mpagado,Master_mpagospesos,Master_mpagosdolares,Master_fechaalta,Master_mconsumototal,Master_cconsumos,Master_cadelantosefectivo,Master_mpagominimo,Visa_delinquency,Visa_status,Visa_mfinanciacion_limite,Visa_Fvencimiento,Visa_Finiciomora,Visa_msaldototal,Visa_msaldopesos,Visa_msaldodolares,Visa_mconsumospesos,Visa_mconsumosdolares,Visa_mlimitecompra,Visa_madelantopesos,Visa_madelantodolares,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria
0,250305092,202101,1,0,0,56,196,3709.29,29071.89,606.04,-1729.79,4375.36,8,1,1,0.0,0.0,2,334155.71,0.0,1274.88,1,441977.68,1,13,91333.25,1,9,25239.66,1,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0,0,0,0,0,2,357620.35,0.0,0,0,0.0,4,4478.19,0,0.0,0,0.0,1,234.60,2,1630.47,0,0.0,0,0.0,0,0.0,8,606.04,0,0,0.0,0,0.0,0,0.0,0,0.0,4,39882.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0,1,54,1,0,0,2,0,4,39882.0,0,0.0,110,0,1,0.0,0.0,691143.79,-879.0,NaN,0.0,0.0,0.0,NaN,NaN,767960.28,NaN,NaN,4.0,0.0,NaN,NaN,4234.0,NaN,NaN,NaN,0.0,0.0,0.0,1265784.30,-1245.0,NaN,21775.40,25542.54,0.0,5091.86,0.0,1406427.00,0.0,0.0,4.0,0.0,-27024.21,0.0,4234.0,5091.86,5.0,0.0,3636.30,CONTINUA
1,250305092,202102,1,0,0,56,197,5885.35,32224.85,753.04,-1097.44,5472.81,8,1,1,0.0,0.0,2,453341.18,0.0,1311.78,1,339546.94,1,9,117194.44,1,9,25530.11,1,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0,0,0,0,0,2,160093.55,0.0,0,0,0.0,4,4768.64,0,0.0,0,0.0,4,8652.54,0,0.00,0,0.0,0,0.0,0,0.0,8,753.04,0,0,0.0,0,0.0,0,0.0,0,0.0,4,18768.0,0,0.0,0,0.0,

In [8]:
df_train[df_train['foto_mes']==np.nan]
df_train['foto_mes'].isna().sum()

0

In [ ]:
import pandas as pd



import numpy as np



df = duckdb.query("SELECT * FROM 'competencia_01_crudo.csv'").df()



start_time = time.time()

# Crear la columna clase_ternaria usando la columna 'foto_mes' basada en la presencia real

df_ternaria= duckdb.query('''

    WITH siguiente_mes AS (

        SELECT

            numero_de_cliente,

            foto_mes,

            -- Obtener los meses siguientes en los que el cliente está presente

            LEAD(foto_mes, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS foto_mes_proximo1,

            LEAD(foto_mes, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS foto_mes_proximo2

        FROM df_parallel

    )

    SELECT

        numero_de_cliente,

        foto_mes,

        CASE

            -- Si el cliente está en los dos meses consecutivos siguientes

            WHEN foto_mes_proximo1 = foto_mes + 1 AND foto_mes_proximo2 = foto_mes + 2 THEN 'CONTINUA'

            -- Si el cliente no está en el siguiente mes

            WHEN foto_mes_proximo1 IS NULL OR foto_mes_proximo1 <> foto_mes + 1 THEN 'BAJA+1'

            -- Si el cliente está en el siguiente mes pero no en el segundo mes consecutivo

            WHEN foto_mes_proximo1 = foto_mes + 1 AND (foto_mes_proximo2 IS NULL OR foto_mes_proximo2 <> foto_mes + 2) THEN 'BAJA+2'

            ELSE NULL

        END AS clase_ternaria

    FROM siguiente_mes

    ORDER BY numero_de_cliente, foto_mes

''')

# Calcular el tiempo de ejecución

end_time = time.time()

execution_time = end_time - start_time

# Mostrar el tiempo de ejecución en segundos

print(f"Tiempo de ejecución: {execution_time:.2f} segundos")


df_ternaria.to_df().to_csv('competencia_ternaria_01.csv', index=False)


In [ ]:
## Clase ternaria

def clase_ternaria_classifier(df):
    df['foto_mes'] = 

In [2]:
df_train = pd.read_csv(r'c:\Users\Admin\Documents\1_Notebook\1_Estudio\1 UBA Maestría DS\1_Especialización\1_Segundo Semestre\DMEyF\datasets\competencia_01.csv')

In [ ]:
X = df_train[df_train['foto_mes'] == 202014]
y = X['clase_ternaria']
X = X.drop(columns=['clase_ternaria'])

ganancia_acierto = 273000
costo_estimulo = 7000

mes_train = 202102
mes_test = 202104

In [6]:
"""FUNCIONES VARIAS"""

def ganancia(model, X, y, prop=1, threshold=0.025):

  class_index = np.where(model.classes_ == "BAJA+2")[0][0]
  y_hat = model.predict_proba(X)

  @np.vectorize
  def ganancia_row(predicted, actual, threshold=0.025):
    return  (predicted >= threshold) * (ganancia_acierto if actual == "BAJA+2" else -costo_estimulo)

  return ganancia_row(y_hat[:,class_index], y).sum() / prop





In [ ]:
sss_opt = ShuffleSplit(n_splits=5, test_size=0.3, random_state=semillas[1])


In [8]:

def objective(trial, X, y, sss):
  criterion = trial.suggest_categorical('criterion', ['gini', 'entropy'])
  max_depth = trial.suggest_int('max_depth', 2, 20)
  min_samples_split = trial.suggest_int('min_samples_split', 2, 200)
  min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 20)
  max_leaf_nodes = trial.suggest_int('max_leaf_nodes', 2, 20)

  def train_and_evaluate(train_index, test_index, X, y):
    m = DecisionTreeClassifier(
        criterion=criterion,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_leaf_nodes=max_leaf_nodes,
        random_state=semillas[0],
    )
    m.fit(X.iloc[train_index],y.iloc[train_index])
    ganancia_value = ganancia(m, X.iloc[test_index], y.iloc[test_index], prop=0.3)
    return ganancia_value

  results = Parallel(n_jobs=-1)(
      delayed(train_and_evaluate)(train_index, test_index, X, y)
      for train_index, test_index in sss.split(X)
  )

  return np.mean(results)

storage_name = "sqlite:///C:\\Users\\Admin\\Documents\\1_Notebook\\1_Estudio\\1 UBA Maestría DS\\1_Especialización\\1_Segundo Semestre\\DMEyF\\Python\\Optuna\\optimization_tree.db"
study_name = "exp_101_decision-tree-opt"

study = optuna.create_study(
    direction="maximize",
    study_name=study_name,
    storage=storage_name,
    load_if_exists=True,
)




[I 2024-09-02 18:01:57,832] A new study created in RDB with name: exp_101_decision-tree-opt


In [ ]:
study.optimize(lambda trial: objective(trial, X, y, sss_opt), n_trials=100)

In [ ]:
study.optimize(lambda trial: objective(trial, X, y, sss_opt), n_trials=100)

In [7]:
# SEEDS 


400009,  500009,  600011,  700021,  800021.

# Analisis exploratorio 